# NeuraVis Business Assistant Agent

An intelligent chatbot powered by OpenAI that represents NeuraVis Technologies.

## 1. Install and Import Dependencies

In [ ]:
# Install required packages (uncomment if running in Colab)
# !pip install openai gradio requests PyPDF2 python-dotenv

In [ ]:
import os
import json
import requests
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import PyPDF2

## 2. Load Environment Variables and Initialize OpenAI

In [ ]:
# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Pushover credentials
pushover_user = os.getenv('PUSHOVER_USER_KEY')
pushover_token = os.getenv('PUSHOVER_TOKEN_KEY')
pushover_url = "https://api.pushover.net/1/messages.json"

## 3. Load Business Information

In [ ]:
def load_business_summary():
    """Load business summary from text file"""
    try:
        with open('me/business_summary.txt', 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading business summary: {e}")
        return ""

def load_business_pdf():
    """Load business information from PDF"""
    try:
        with open('me/about_business.pdf', 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return ""

# Load business information
business_summary = load_business_summary()
business_pdf_content = load_business_pdf()

print("Business Summary Loaded:")
print(business_summary[:200] + "...")

## 4. Define Tool Functions

In [ ]:
def push(message):
    """Send push notification via Pushover"""
    print(f"📱 Push: {message}")
    try:
        payload = {
            "user": pushover_user,
            "token": pushover_token,
            "message": message
        }
        response = requests.post(pushover_url, data=payload)
        return response.status_code == 200
    except Exception as e:
        print(f"Push notification error: {e}")
        return False

def record_customer_interest(email, name, message):
    """Record customer interest - logs lead information"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"[LEAD] {timestamp} | Name: {name} | Email: {email} | Message: {message}"
    
    # Print to console (visible in Colab)
    print("\n" + "="*60)
    print("🎯 NEW LEAD CAPTURED!")
    print("="*60)
    print(f"Name:    {name}")
    print(f"Email:   {email}")
    print(f"Message: {message}")
    print(f"Time:    {timestamp}")
    print("="*60 + "\n")
    
    # Log to file
    try:
        with open('customer_leads.log', 'a', encoding='utf-8') as f:
            f.write(log_message + "\n")
    except Exception as e:
        print(f"File logging error: {e}")
    
    # Send push notification
    push_msg = f"New Lead: {name} ({email}) - {message[:50]}..."
    push(push_msg)
    
    return f"Thank you {name}! We've recorded your interest and will contact you at {email} soon."

def record_feedback(question):
    """Record unanswered questions or feedback"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"[FEEDBACK] {timestamp} | Question: {question}"
    
    # Print to console (visible in Colab)
    print("\n" + "="*60)
    print("💭 FEEDBACK RECORDED!")
    print("="*60)
    print(f"Question: {question}")
    print(f"Time:     {timestamp}")
    print("="*60 + "\n")
    
    # Log to file
    try:
        with open('customer_feedback.log', 'a', encoding='utf-8') as f:
            f.write(log_message + "\n")
    except Exception as e:
        print(f"File logging error: {e}")
    
    # Send push notification
    push_msg = f"Unanswered Question: {question[:100]}..."
    push(push_msg)
    
    return "I've recorded your question for our team to review. Is there anything else I can help you with?"

# Test the push function
push("NeuraVis Business Agent initialized!")

## 5. Define OpenAI Tools Schema

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Record a customer's contact information and interest in NeuraVis services. Use when customer wants to learn more, get contacted, or shows interest in our services.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Customer's email address"
                    },
                    "name": {
                        "type": "string",
                        "description": "Customer's full name"
                    },
                    "message": {
                        "type": "string",
                        "description": "Customer's message, interest area, or specific needs"
                    }
                },
                "required": ["email", "name", "message"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Record questions that you cannot answer or customer feedback. Use when you don't have enough information to answer a question.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question or feedback from the customer"
                    }
                },
                "required": ["question"]
            }
        }
    }
]

print("Tools defined:", [tool['function']['name'] for tool in tools])

## 6. Create System Prompt

In [ ]:
system_prompt = f"""You are the AI assistant for NeuraVis Technologies, a company specializing in ethical and transparent AI solutions.

BUSINESS INFORMATION:
{business_summary}

ADDITIONAL DETAILS:
{business_pdf_content if business_pdf_content else 'See summary above'}

YOUR ROLE:
1. Stay in character as NeuraVis's helpful and professional AI representative
2. Answer questions about NeuraVis's mission, services, team, and values
3. When customers show interest, encourage them to share their contact information (name, email, and their needs)
4. Use the record_customer_interest function when they provide their contact details
5. If you don't know the answer to a question, use the record_feedback function to log it
6. Be friendly, concise, and helpful
7. Emphasize NeuraVis's commitment to ethical AI and human-centric solutions

IMPORTANT:
- Always try to answer based on the business information provided
- If asked about pricing, technical details, or specific implementations you're unsure about, record it as feedback
- Actively encourage interested visitors to leave their contact information
"""

print("System prompt created successfully")

## 7. Implement Chat Logic with Tool Calling

In [ ]:
def call_tool(tool_name, arguments):
    """Execute the requested tool function"""
    if tool_name == "record_customer_interest":
        return record_customer_interest(
            email=arguments.get("email"),
            name=arguments.get("name"),
            message=arguments.get("message")
        )
    elif tool_name == "record_feedback":
        return record_feedback(
            question=arguments.get("question")
        )
    else:
        return "Unknown tool"

def chat_with_agent(message, history):
    """Main chat function for Gradio interface"""
    # Convert Gradio history format to OpenAI messages format
    messages = [{"role": "system", "content": system_prompt}]
    
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if assistant_msg:
            messages.append({"role": "assistant", "content": assistant_msg})
    
    messages.append({"role": "user", "content": message})
    
    try:
        # Initial API call
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        # Check if the model wants to call a tool
        if response_message.tool_calls:
            # Add assistant's response to messages
            messages.append(response_message)
            
            # Process each tool call
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"Calling tool: {function_name} with args: {function_args}")
                
                # Execute the tool
                function_response = call_tool(function_name, function_args)
                
                # Add tool response to messages
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                })
            
            # Get final response from the model
            second_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            
            return second_response.choices[0].message.content
        
        else:
            # No tool call needed, return the response
            return response_message.content
            
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        print(error_msg)
        return "I apologize, but I'm experiencing technical difficulties. Please try again."

print("Chat function ready")

## 8. Create Gradio Interface

In [ ]:
def get_logs():
    """Get all logs for display"""
    leads_log = ""
    feedback_log = ""
    
    # Get leads
    try:
        with open('customer_leads.log', 'r', encoding='utf-8') as f:
            leads_log = f.read()
    except FileNotFoundError:
        leads_log = "No leads yet."
    
    # Get feedback
    try:
        with open('customer_feedback.log', 'r', encoding='utf-8') as f:
            feedback_log = f.read()
    except FileNotFoundError:
        feedback_log = "No feedback yet."
    
    return f"📊 CUSTOMER LEADS:\n{leads_log}\n\n💭 FEEDBACK:\n{feedback_log}"

print("Log viewer function created")

In [ ]:
# Create Gradio ChatInterface
demo = gr.ChatInterface(
    fn=chat_with_agent,
    title="NeuraVis Technologies - AI Assistant",
    description="""Welcome to NeuraVis Technologies! 🤖
    
I'm here to help you learn about our ethical AI solutions and services. Feel free to ask me:
- About our mission and values
- What services we offer
- Our team and expertise
- How we can help your organization

If you're interested in working with us, I can collect your contact information!""",
    examples=[
        "What is NeuraVis Technologies?",
        "What services do you offer?",
        "Tell me about your team",
        "How can NeuraVis help my business?",
        "I'm interested in your AI consulting services"
    ],
    theme=gr.themes.Soft()
)

print("Gradio interface created")

## 9. Launch the Chatbot

In [ ]:
# Launch the interface
if __name__ == "__main__":
    demo.launch(share=True)

## 10. View Logs (Optional)

In [ ]:
# View customer leads
try:
    with open('customer_leads.log', 'r', encoding='utf-8') as f:
        print("=== CUSTOMER LEADS ===")
        print(f.read())
except FileNotFoundError:
    print("No customer leads yet")

print("\n" + "="*50 + "\n")

# View feedback
try:
    with open('customer_feedback.log', 'r', encoding='utf-8') as f:
        print("=== CUSTOMER FEEDBACK ===")
        print(f.read())
except FileNotFoundError:
    print("No feedback yet")